<a href="https://colab.research.google.com/github/slymnk10/Phishing-Email/blob/main/phishing_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Başlangıç

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # drive klasörünü colab'a bağlıyor

Mounted at /content/drive


In [ ]:
veri_yolu = '/content/drive/My Drive/Phishing_Email.csv'
model_yolu = '/content/drive/My Drive/YapayZeka/phishing_ai.keras' # kaydedilecek model yolu


#Kütüphaneler


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split # Veriyi eğitim ve test olarak bölmek için.

# Keras bileşenlerini TensorFlow üzerinden içe aktar
from tensorflow.keras.preprocessing.text import Tokenizer # metin verisini sayıya çevirmek için
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense # model mimarisi kurmak için
from tensorflow.keras.utils import to_categorical

import joblib
import tensorflow as tf


#Veri İşlenmesi



In [ ]:
mail_verisi = pd.read_csv(veri_yolu) # veri dosyasını oku
mail_verisi = mail_verisi.where((pd.notnull(mail_verisi)), '') # eğer eksik veriler varsa boş string ile doldur

mail_verisi.loc[mail_verisi['Email Type']== 'Safe Email' , 'Email Type']= 0 # sınıfları sayısal olarak kullanmak için
mail_verisi.loc[mail_verisi['Email Type']== 'Phishing Email' , 'Email Type']= 1

X = mail_verisi['Email Text']
Y = mail_verisi['Email Type']
Y = to_categorical(Y) # softmax için gerekli



#Tokenizer

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X) # fonksiyon kelimeleri tanır
sequences = tokenizer.texts_to_sequences(X) # fonksiyon her kelimeyi bir sayıya çevirir

word_index = tokenizer.word_index
max_sequence_length = 100
veri = pad_sequences(sequences, maxlen=max_sequence_length) # tüm e postalar 100 kelimelik sabit dizilere dönüştürür

#Test Train Verisi

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(veri, Y, test_size=0.2, random_state= 3) # Verinin %80’i eğitim, %20’si test için ayrılır.

#Model Oluşturma

In [ ]:
model = Sequential()
model.add(Embedding(len(word_index) + 1, 128, input_length=max_sequence_length)) # Embedding : Kelimeleri 128 boyutlu vektöre çevirir
model.add(SimpleRNN(64)) # SimpleRNN : Zaman serisi gibi metin sırasını işler
model.add(Dense(2, activation='softmax')) #Dense : İki sınıfa tahmin yapar (softmax ile)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


#Model Derleme

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # adam : optimizasyon algoritması ,
#categorical_crossentropy: çok sınıflı sınıflandırma kaybı


#Eğitim Süreci

In [ ]:
model.fit(X_train, Y_train, epochs=5, batch_size=32, validation_data=(X_test, Y_test))
#epochs=5: verilerle 5 kez eğitim
#batch_size=32: her adımda 32 örnekle öğrenir

Epoch 1/5
467/467 ━━━━━━━━━━━━━━━━━━━━ 177s 369ms/step - accuracy: 0.7954 - loss: 0.4075 - val_accuracy: 0.9466 - val_loss: 0.1483
Epoch 2/5
467/467 ━━━━━━━━━━━━━━━━━━━━ 192s 348ms/step - accuracy: 0.9752 - loss: 0.0571 - val_accuracy: 0.9592 - val_loss: 0.1303
Epoch 3/5
467/467 ━━━━━━━━━━━━━━━━━━━━ 204s 353ms/step - accuracy: 0.9845 - loss: 0.0331 - val_accuracy: 0.9450 - val_loss: 0.1568
Epoch 4/5
467/467 ━━━━━━━━━━━━━━━━━━━━ 214s 378ms/step - accuracy: 0.9856 - loss: 0.0249 - val_accuracy: 0.9091 - val_loss: 0.2499
Epoch 5/5
467/467 ━━━━━━━━━━━━━━━━━━━━ 189s 351ms/step - accuracy: 0.9887 - loss: 0.0235 - val_accuracy: 0.9550 - val_loss: 0.1672


#Model Kayıt

In [ ]:
import os
os.makedirs(os.path.dirname(model_yolu), exist_ok=True)

model.save(model_yolu)
#Eğitilen modeli .keras dosyası olarak Google Drive’a kaydeder

#Model Değerlendirme

In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test)
print("Test verilerin doğruluğu : " , accuracy)
#Test verileriyle başarı oranını ölçer

117/117 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9555 - loss: 0.1390
Test verilerin doğruluğu :  0.9530830979347229


#Gerçek Zamanlı Test

#Model Yüklenmesi

In [ ]:
model = tf.keras.models.load_model(model_yolu)

#Test Aşaması

In [ ]:
while True:
  input_mail = input("E - posta mesajını giriniz : ")
  input_sequence = tokenizer.texts_to_sequences([input_mail])
  input_data = pad_sequences(input_sequence, maxlen=max_sequence_length)
  tahmin = model.predict(input_data)

  if np.argmax(tahmin) == 1:
    print("Bu bir kimlik avı e-postası olabilir.")
  else:
    print("Bu güvenli bir e-postadır.")
  oran = tahmin[0][np.argmax(tahmin)]
  print("Eminlik oranı : ",oran)


KeyboardInterrupt: Interrupted by user